# Cyclic Polling System with K-Limited Service

This example demonstrates a cyclic polling system with k-limited service discipline. In k-limited service, the server serves at most k jobs per visit to each queue, where k=1 in this case.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
val model = Network("M[2]/M[2]/1-KLimited")

// Block 1: nodes
val source = Source(model, "mySource")
val queue = Queue(model, "myQueue", SchedStrategy.POLLING)
val sink = Sink(model, "mySink")

In [ ]:
// Block 2: classes with moderate traffic loads
val oclass1 = OpenClass(model, "myClass1")
source.setArrival(oclass1, Exp(0.2))  // λ₁ = 0.2
queue.setService(oclass1, Exp(1.0))   // μ₁ = 1.0, so E[S₁] = 1.0

val oclass2 = OpenClass(model, "myClass2")
source.setArrival(oclass2, Exp(0.3))  // λ₂ = 0.3
queue.setService(oclass2, Exp(1.5))   // μ₂ = 1.5, so E[S₂] ≈ 0.667

// Configure k-limited polling system (k=1)
queue.setPollingType(PollingType.KLIMITED, 1)  // Serve at most 1 job per visit
queue.setSwitchover(oclass1, Exp(1.0))          // Switchover to class 1: Exp(1.0)
queue.setSwitchover(oclass2, Immediate())       // Switchover to class 2: Immediate

In [ ]:
// Block 3: topology
val P = model.initRoutingMatrix()
P.addRoute(oclass1, source, queue, sink)
P.addRoute(oclass2, source, queue, sink)
model.link(P)

In [ ]:
// Calculate system parameters for k-limited polling
val lambda1 = 0.2
val lambda2 = 0.3
val mu1 = 1.0
val mu2 = 1.5
val k = 1  // k-limited parameter
val switchover1Mean = 1.0  // Mean switchover time to class 1
val switchover2Mean = 0.0  // Immediate switchover to class 2

val rho1 = lambda1 / mu1  // ρ₁ = 0.2 / 1.0 = 0.2
val rho2 = lambda2 / mu2  // ρ₂ = 0.3 / 1.5 = 0.2
val rhoTotal = rho1 + rho2
val totalArrivalRate = lambda1 + lambda2

println("=== K-Limited Polling System Analysis (k=$k) ===")
println("Class 1: λ₁ = $lambda1, μ₁ = $mu1, ρ₁ = $rho1")
println("Class 2: λ₂ = $lambda2, μ₂ = $mu2, ρ₂ = $rho2")
println("Total service utilization: ρ = $rhoTotal")
println("K-limited parameter: k = $k (serve at most $k job per visit)")
println("\nSwitchover Configuration:")
println("- Class 2 → Class 1: Exponential with mean $switchover1Mean")
println("- Class 1 → Class 2: Immediate (no switchover time)")
println("\nTotal arrival rate: λ = $totalArrivalRate")
println("System is ${if (rhoTotal < 1.0) "stable" else "unstable"} (ρ = $rhoTotal)")

In [ ]:
// Solve with MVA (solution is approximate for k-limited polling)
try {
    val mvaSolver = MVA(model)
    val avgTableMVA = mvaSolver.avgTable
    println("\nMVA Results:")
    avgTableMVA.print()
} catch (e: Exception) {
    println("\nMVA failed for this polling model: ${e.message}")
    println("This is expected as MVA may not support all polling configurations.")
}

In [ ]:
// Try solving with JMT
try {
    val jmtOptions = JMT.defaultOptions()
    jmtOptions.samples = 100000
    jmtOptions.seed = 23000
    
    val jmtSolver = JMT(model, jmtOptions)
    val avgTableJMT = jmtSolver.avgTable
    println("\nJMT Results:")
    avgTableJMT.print()
} catch (e: Exception) {
    println("\nJMT failed for this polling model: ${e.message}")
    println("This is expected as JMT may not support all polling configurations.")
}

In [ ]:
// Analyze k-limited service characteristics
println("\n=== K-Limited Service Discipline Analysis ===")
println("Model: ${model.name}")
println("Polling Type: K-Limited (k=$k)")
println("\nK-Limited Service Rules:")
println("- Server serves at most k=$k jobs per visit to each queue")
println("- If queue has ≤k jobs, serves all; if >k jobs, serves exactly k")
println("- Provides bounded service times and fairness")
println("- Balances efficiency and response time predictability")
println("\nWith k=1 (1-Limited):")
println("- Serves exactly 1 job per visit (if queue non-empty)")
println("- Maximum fairness but potentially high overhead")
println("- Frequent switching between queues")
println("\nAsymmetric Switchover Impact:")
println("- No delay switching from Class 1 to Class 2")
println("- Exponential delay (mean=1.0) switching from Class 2 to Class 1")
println("- Creates slight bias toward Class 2 service")
println("\nLoad Balance:")
println("- Both classes have equal utilization (ρ₁ = ρ₂ = 0.2)")
println("- Different arrival and service rates but same load")
println("- Class 1: Lower arrival rate, slower service")
println("- Class 2: Higher arrival rate, faster service")
println("\nExpected Performance:")
println("- Response times should be relatively balanced")
println("- Class 1 may have slightly higher response time due to switchover delays")
println("- System stability is good with total ρ = $rhoTotal")

This example demonstrates:

1. **K-Limited Polling (k=1)**: 
   - Server serves at most 1 job per visit to each queue
   - Provides maximum fairness among job classes
   - Also known as "1-limited" or "round-robin" service
2. **Balanced Load Design**:
   - Both classes have equal utilization (ρ = 0.2)
   - Achieved through different combinations of λ and μ
   - Class 1: Lower λ, lower μ
   - Class 2: Higher λ, higher μ
3. **Asymmetric Switchover Times**:
   - Immediate switching from Class 1 to Class 2
   - Exponential delay when switching from Class 2 to Class 1
   - Creates potential bias in service order
4. **Performance Characteristics**:
   - Highly predictable service patterns
   - Good fairness properties
   - Higher switching overhead compared to exhaustive/gated
   - Suitable for real-time systems requiring bounded response times

K-limited polling is widely used in:
- Time-division systems with strict fairness requirements
- Real-time communication protocols
- Systems where response time predictability is more important than throughput
- Fair queueing implementations in network routers